In [23]:
import numpy as np
import pandas as pd
import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.feature_extraction.text import CountVectorizer

# Dataset uploading

In [3]:
"""
Note: Did not test on kaggle yet.
"""
import os 

DATASET_PATHS = {
    "local": {
        "train": "../../datasets/train_set.csv",
        "test": "../../datasets/test_set.csv"
    },
    "kaggle": {
        "train": "/kaggle/input/python-codes-time-complexity/train_set.csv",
        "test": "/kaggle/input/python-codes-time-complexity/test_set.csv"
    }
}

def upload_datasets(dataset_paths=DATASET_PATHS):
    for path in dataset_paths:
        if os.path.exists(dataset_paths[path]['train']) and os.path.exists(dataset_paths[path]['test']):
            return dataset_paths[path]['train'], dataset_paths[path]['test']

    return FileNotFoundError(f"Datasets do not exist in the current paths: {dataset_paths}")
            

train_set_path, test_set_path = upload_datasets()

# Dataset Uploading

In [24]:
train_set = pd.read_csv(train_set_path)
test_set = pd.read_csv(test_set_path)

X_train = train_set['code']
y_train = train_set['complexity']

X_test = test_set['code']
y_test = test_set['complexity']

# Metrics

## Hierarchy Complexity Score

### Ordering labels by Hierarchy

In [5]:
LABELS_HIERARCHY = {
    'constant': 1,
    'logn': 2,
    'linear': 3,
    'nlogn': 4,
    'quadratic': 5,
    'cubic': 6,
    'np': 7
}

N_CLASSES = len(LABELS_HIERARCHY)

### Writing the custom metric hc_score

In [8]:
def hc_score(y_true, y_pred, n_classes=N_CLASSES):
    assert len(y_true) == len(y_pred), f"The amount of y_true labels: {len(y_true)} does not equal to the amount of y_pred: {len(y_pred)}."
    
    n_samples = len(y_true)
    return (np.sum(np.abs(y_pred - y_true)) / n_classes) / n_samples

hierarchy_score = make_scorer(hc_score)

### All metrics

In [21]:
metrics = {'Accuracy': 'accuracy', 'F1_MACRO': 'f1_macro', 'hc_score': hierarchy_score}

# Tokenizer

In [25]:
tokenizer = CountVectorizer()

X_train = tokenizer.fit_transform(X_train)
X_test = tokenizer.transform(X_test)

# Model

In [32]:
model = LogisticRegression(max_iter=10000, random_state=2025)

scores = cross_validate(model, X_train, y_train, scoring=metrics, cv=5)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:1000: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pandas/core/ops/array_ops.py", line 218, in _na_arithmetic_op
    result = func(left, right)
             ^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pandas/core/computation/expressions.py", line 242, in evaluate
    return _evaluate(op, op_str, a, b)  # type: ignore[misc]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pandas/core/computation/expressions.py", line 73, in _evaluate_standard
    return op(a, b)
           ^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12